# Stereo Images Parallax Estimation using RIPOC image correlation check

In [ ]:
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import math 
import numpy as np
import os

from skimage.color import rgba2rgb, rgb2gray
from skimage.filters import meijering, sato, frangi, hessian
from skimage.util import dtype_limits
# from matplotlib.image import imread
from mpl_toolkits.mplot3d import Axes3D

## affine変換による並行回転拡大
##########
def image_affine(img_src, x, y, angle = 0, scale = 1.0):
    h, w = img_src.shape[:2]

    # 回転中心
    center = (w/2, h/2)
    # 回転・拡大行列 (angle は degree(度), scale は 1.0 = 100%として指定)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    # 並行移動成分を追加
    M[0][2] -= x
    M[1][2] -= y
    img_dst = cv2.warpAffine(img_src, M, (w, h))
    return img_dst
    
## convert gray image by percentile
##########
def gray_image(inimg, percentile = -1):
    if(inimg.ndim == 3):
        srcimg = cv2.cvtColor(inimg, cv2.COLOR_BGR2GRAY)
    else:
        srcimg = inimg.copy()
    dstimg = np.zeros(srcimg.shape)
    maxvalue = np.max(srcimg)
    minvalue = np.min(srcimg)
    if(percentile > 0):
        maxvalue = np.percentile(srcimg, percentile)
        srcimg = np.clip(srcimg, a_min = minvalue, a_max = maxvalue)
        # print("max = %.1f, min = %.1f" % (maxvalue, minvalue))
    if(maxvalue <= minvalue):
        maxvalue = 1
        minvalue = 0
    dstimg = cv2.convertScaleAbs(srcimg - minvalue, alpha = 255 / (maxvalue - minvalue), beta = 0)
    dstimg = dstimg.astype(np.uint8)
    return dstimg

## color 画像に変換 (convert color image)
##########
def color_image(srcimg):
    # print(img.shape)
    
    if(srcimg.ndim == 3):
        dstimg = srcimg
    else:
        gray = srcimg
        if(srcimg.dtype != "uint8"):
            gray = gray_image(srcimg)
        dstimg = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    return dstimg

## get keypoints
##########
def get_keypoints(img, flag_negative=False, flag_resize=True, flag_edge=True):
    
    ## size and planes
    height, width, planes = img.shape
    
    ## convert gray scale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if(flag_negative):
        img_gray = 255 - img_gray
    
    ## Hisgram equalization
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    img_clahe = clahe.apply(img_gray)
    # img_clahe = cv2.equalizeHist(img_gray)
    # img_clahe = img_gray.copy()
    
    ## Resize
    img_resized = img_clahe.copy()
    if(flag_resize):
        img_resized = cv2.resize(img_resized, (int(width / 2), int(height / 2)))
    
    ## DeNoising
    # NOISEBLOCKSIZE = 20
    NOISEBLOCKSIZE = 30
    img_denoised = cv2.fastNlMeansDenoising(img_resized, h=NOISEBLOCKSIZE)
    # img_denoised = img_resized.copy()
    
    ## filtered
    img_filtered = img_denoised.copy()
    if(flag_edge):
        # img_filtered = gray_image(frangi(img_denoised))
        # img_filtered = clahe.apply(gray_image(frangi(img_denoised)))
        # img_filtered = gray_image(meijering(img_denoised))
        # img_filtered = clahe.apply(gray_image(meijering(img_denoised)))
        # img_filtered = gray_image(sato(img_denoised))
        # img_filtered = cv2.Canny(img_denoised, 0, 200)
        # img_filtered = cv2.morphologyEx(cv2.Canny(img_denoised, 0, 100), cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
        # img_filtered = cv2.Laplacian(img_denoised, -1, ksize=3) 
        img_filtered = clahe.apply(cv2.Laplacian(img_denoised, -1, ksize=3))
    
    ## Initiate SIFT or AKAZE detector *select one*
    ## find the keypoints and descriptors with SIFT or AKAZE
    # detector = cv2.AKAZE_create()
    # detector = cv2.BRISK_create()
    # detector = cv2.FastFeatureDetector_create() # not implemented
    # detector = cv2.KAZE_create()
    detector = cv2.ORB_create()
    # detector = cv2.SIFT_create(1000)
    # detector = cv2.SIFT_create()
    # detector = cv2.xfeatures2d.SURF_create() # patented, not implemented
    # detector = cv2.xfeatures2d.LATCH_create() # not implemented
    kp, des = detector.detectAndCompute(img_filtered, None)
    img_key = cv2.drawKeypoints(img_filtered, kp, None, flags=4)
    
    return kp, des, img_resized, img_denoised, img_clahe, img_filtered, img_key

## RIPOC function
## Obtain parameters for translation (x, y), rotation (angle), and scaling (scale)
## with response, and some processing images
##########
def RIPOCfunc(imgR, imgM, flag_Debug=True):
    ## flag_Debug = True  ## display step-by-step images for debug

    # 画像読み込み
    ref_img = imgR
    tgt_img = imgM

    # Numpy配列に変換
    ref_img_gray = np.array(ref_img,dtype=np.float64)
    tgt_img_gray = np.array(tgt_img,dtype=np.float64)

    # 画像サイズ定義
    h, w = ref_img_gray.shape
    center = (w/2, h/2)

    # 窓関数定義
    hunning_y = np.hanning(h)
    hunning_x = np.hanning(h)
    hunning_w = hunning_y.reshape(h, 1)*hunning_x

    # フーリエ変換
    ref_img_fft = np.fft.fftshift(np.log(np.abs(np.fft.fft2(ref_img_gray*hunning_w))))
    tgt_img_fft = np.fft.fftshift(np.log(np.abs(np.fft.fft2(tgt_img_gray*hunning_w))))

    # 対数極座標変換 (lanczos法補間)
    l = np.sqrt(w*w + h*h)
    m = l/np.log(l)
    flags = cv2.INTER_LANCZOS4 + cv2.WARP_POLAR_LOG
    ref_img_pol = cv2.warpPolar(ref_img_fft, (w, h), center, m, flags)
    tgt_img_pol = cv2.warpPolar(tgt_img_fft, (w, h), center, m, flags)

    # 位相限定相関法
    (x, y), response = cv2.phaseCorrelate(ref_img_pol, tgt_img_pol, hunning_w)

    # affine変換による平行移動
    angle = y*360/h
    scale = (np.e)**(x/m)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    tgt_img_affine = cv2.warpAffine((tgt_img_gray), M, (w, h))

    # 位相限定相関法
    (x, y), response = cv2.phaseCorrelate(ref_img_gray, tgt_img_affine)

    #位相限定相関法の返り値から画像を生成
    M[0][2] -= x
    M[1][2] -= y
    dst = cv2.warpAffine(tgt_img, M, (w, h))

    if(flag_Debug):
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.ylabel('input')
        plt.xlabel('reference')
        plt.imshow(ref_img)
        plt.subplot(1, 2, 2)
        plt.xlabel('target')
        plt.imshow(tgt_img)
        plt.show()
        
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.ylabel('FFT')
        plt.xlabel('reference')
        plt.imshow(ref_img_fft)
        plt.subplot(1, 2, 2)
        plt.xlabel('target')
        plt.imshow(tgt_img_fft)
        plt.show()

        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.ylabel('polar')
        plt.xlabel('reference')
        plt.imshow(ref_img_pol)
        plt.subplot(1, 2, 2)
        plt.xlabel('target')
        plt.imshow(tgt_img_pol)
        plt.show()
        
        px = pz = np.linspace(-100, 100, h)
        px , pz = np.meshgrid(px, pz)
        img = ref_img_pol
        py = img
        fig = plt.figure(figsize=(10, 5))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter3D(np.ravel(px), np.ravel(pz), np.ravel(py),s=10,marker='.',c=img)
        ax.set_title("image")
        plt.show()

    return x, y, angle, scale, response, tgt_img_affine, dst




# RDS to Anaglyph 3D - RDS parallax estimation

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

##
## draw triangle
##
def triangle_draw(img, c = (0, 0, 0), ox = 70, oy = 70, d = 0, s = 10, text_flag = False):
    ax =  s * np.cos(( 30 + d) * np.pi / 180) + ox
    ay = -s * np.sin(( 30 + d) * np.pi / 180) + oy
    bx =  s * np.cos((-30 + d) * np.pi / 180) + ox
    by = -s * np.sin((-30 + d) * np.pi / 180) + oy
    cx =  (s) * np.cos(d * np.pi / 180) + ox
    cy = -(s) * np.sin(d * np.pi / 180) + oy
    pts = np.array([(int(ox), int(oy)), (int(ax), int(ay)), (int(bx), int(by))])
    img = cv2.polylines(img, [pts], True, c, thickness=3)
    img = cv2.fillConvexPoly(img, pts, (0, 0, 0))
    if(text_flag):
        fontScale = 0.5
        fontColor = (255, 255, 255)
        font = cv2.FONT_HERSHEY_SIMPLEX
        text = '%03d' % (d)
        img = cv2.putText(img, text, (int(cx + 1), int(cy + 1)), font, fontScale, (0, 0, 0), 2, cv2.LINE_AA)
        img = cv2.putText(img, text, (int(cx + 0), int(cy + 0)), font, fontScale, fontColor, 1, cv2.LINE_AA)
    return img

## 【ガンマ補正の公式】
## Gamma correction
##   Y = 255(X/255)**(1/γ)
## 【γの設定方法】
## How to set Y parameter
##   ・γ>1の場合：画像が明るくなる / case Y>1: image become brighter
##   ・γ<1の場合：画像が暗くなる / case Y<1: image become darker
def gamma_correction(img, gamma = 1.6):
    # ガンマ変換用の数値準備 
    # gamma     = 3.0                               # γ値を指定
    img2gamma = np.zeros((256,1),dtype=np.uint8)  # ガンマ変換初期値
    # 公式適用
    for i in range(256):
        img2gamma[i][0] = 255 * (float(i)/255) ** (1.0 /gamma)
    # 読込画像をガンマ変換
    gamma_img = cv2.LUT(img, img2gamma)
    return gamma_img

##
## 画像の切り出し
## crop image
##
def image_crop(img_src, x, y, l):
    h, w = img_src.shape[:2]
    cx = int(x)
    cy = int(y)
    cx1 = int(cx - l/2)
    cx2 = int(cx + l/2)
    cy1 = int(cy - l/2)
    cy2 = int(cy + l/2)
    if(cx1 < 0):
        cx1 = 0
        cx2 = cx1 + l
    if(cx2 >= w):
        cx2 = w - 1
        cx1 = cx2 - l
    if(cy1 < 0):
        cy1 = 0
        cy2 = cy1 + l
    if(cy2 >= h):
        cy2 = h - 1
        cy1 = cy2 - l
    img_dst = img_src[cy1:cy2, cx1:cx2]
    return img_dst

##
## affine変換による並行回転拡大
## Parallel movement, rotation, and zoom by affine transformation
##
def image_affine(img_src, x, y, angle = 0, scale = 1.0):
    h, w = img_src.shape[:2]

    # 回転中心 center of rotation
    center = (w/2, h/2)
    # 回転・拡大行列 (angle は degree(度), scale は 1.0 = 100%として指定)
    # rotation and zoom matrix (unit of angle is degree, scale 1.0 means 100%)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    # 並行移動成分を追加
    # Added parallel moving components
    M[0][2] -= x
    M[1][2] -= y
    img_dst = cv2.warpAffine(img_src, M, (w, h))
    return img_dst

##########################################################################
### Main routine
###

## when image saves, set True
flag_image_save = False

## to find how many parallax (focus candidate) 
max_n_paralax = 1

## set RDS file name
rds_file = 'images/RDS/16_100_ROBO.png'
#rds_file = 'images/RDS/20_150_CUBE.png'
#rds_file = 'images/RDS/25_160_EARTH.png'

## If the parallax estimation fails, try changing the window size (l) and height position (r). 
## window size of block matching area
l = 120
## height position (ratio) of sliding window
r = 1/2

img_src = cv2.imread(rds_file)
h, w = img_src.shape[:2]
y1 = int(h * r)

flag_resize = False
flag_negative = False
flag_edge = False

#for x1 in range(int(l/2), int(w - l/2) - 1):
dif = []
mov = []
res = []
zero_dif = []
zero_mov = []
zero_res = []

for x1 in [int(w/2)]:
    imgR_org = image_crop(img_src, x1, y1, l)
    GRAY_PERCENTILE = 98
    imgR_gray = gray_image(imgR_org, GRAY_PERCENTILE)
    imgR_color = color_image(imgR_gray)
    kpR, desR, imgR_resized, imgR_denoise, imgR_clahe, imgR_filtered, imgR_key = get_keypoints(imgR_color, flag_negative, flag_resize, flag_edge)

    y2 = y1
    for x2 in range(x1 + 1, int(w - l/2)):
        imgM_org = image_crop(img_src, x2, y2, l)
        # imgM_org = cv2.resize(imgM_org, imgR_org.shape)
        GRAY_PERCENTILE = 98
        imgM_gray = gray_image(imgM_org, GRAY_PERCENTILE)
        imgM_color = color_image(imgM_gray)
        kpM, desM, imgM_resized, imgM_denoise, imgM_clahe, imgM_filtered, imgM_key = get_keypoints(imgM_color, flag_negative, flag_resize, flag_edge)
        
        x, y, angle, scale, response, imgM_affine, imgM_result = RIPOCfunc(imgR_filtered, imgM_filtered, flag_Debug=False)
        # print("refer(%d, %d), differ=%d, move(%.1f, %.1f), angle=%.3f, scale=%.3f, response=%.3f" % (x1, y1, x2 - x1, x, y, angle, scale, response))
        dif.append(x2 - x1)
        mov.append(x)
        res.append(response)
        
        #if(np.abs(x) < 1):
        #    zero_mov.append(x)
        #    zero_dif.append(x2 - x1)
        #    zero_res.append(response)
        
        cv2.imshow('Reference'    , imgR_filtered)
        cv2.imshow('Moving Target', imgM_filtered)
        cv2.waitKey(1)

for i in range(len(mov) - 1):
    if(((mov[i] >= 0 and mov[i + 1] <= 0) or 
         (mov[i] <= 0 and mov[i + 1] >= 0)) and 
        np.abs(mov[i] - mov[i + 1]) < 5 and 
        res[i] > 0.1):
        zero_mov.append(mov[i])
        zero_dif.append(dif[i])
        zero_res.append(res[i])

plt.figure(figsize=(10, 4))
plt.subplot(2, 1, 1)
plt.xlabel('Difference [pix]')
plt.ylabel('Moving [pix]')
# plt.ylim(0, 1.2)
# plt.xlim(-1,1)
ymax = max(np.abs(mov))
ymax = 60 if(ymax > 60) else ymax
#ymax = 200 if(ymax > 200) else ymax
plt.xlim(0, int(w / 2))
plt.ylim(-ymax, ymax)
plt.plot(dif, mov)
plt.plot(zero_dif, zero_mov, 'o')
plt.grid()
plt.subplot(2, 1, 2)
plt.xlabel('Difference [pix]')
plt.ylabel('Response')
plt.xlim(0, int(w / 2))
# plt.ylim(0, 1.2)
plt.plot(dif, res)
plt.plot(zero_dif, zero_res, 'o')
plt.grid()
plt.show()

print("diff    :", zero_dif)
print("moving  :", zero_mov)
print("response:", zero_res)

color = [(128, 128, 128),
         (255, 128, 128),
         (128, 128, 255),
         (255, 128, 255),
         (128, 192, 128),
         (255, 255, 128),
         (128, 255, 255),
         (255, 255, 255)]

if(len(zero_dif) > 0):
    n = 0
    c = 1
    pd_list = []
    img_dot = img_src.copy()
    for d in zero_dif:
        x1 = int(w / 2 - d / 2)
        x2 = int(w / 2 + d / 2)
        y1 = 32 if (n % 2 == 0) else (h - 32) # int(h/2)
        y2 = y1
        if(n % 4 < 2):
            cv2.circle(img_dot, (x1, y1), 8, color[c], thickness= 3)
            cv2.circle(img_dot, (x1, y1), 8, (0,0,0) , thickness=-1)
            cv2.circle(img_dot, (x2, y2), 8, color[c], thickness= 3)
            cv2.circle(img_dot, (x2, y2), 8, (0,0,0) , thickness=-1)
        else:
            triangle_draw(img_dot, c=color[c], ox=x1, oy=y1-8, d=-90, s=16) 
            triangle_draw(img_dot, c=color[c], ox=x2, oy=y2-8, d=-90, s=16) 
            #cv2.rectangle(img_dot, (x1 - 8, y1 - 8), (x1 + 8, y1 + 8), color[c] , thickness=  3) 
            #cv2.rectangle(img_dot, (x1 - 8, y1 - 8), (x1 + 8, y1 + 8), (0, 0, 0), thickness= -1) 
            #cv2.rectangle(img_dot, (x2 - 8, y2 - 8), (x2 + 8, y2 + 8), color[c] , thickness=  3) 
            #cv2.rectangle(img_dot, (x2 - 8, y2 - 8), (x2 + 8, y2 + 8), (0, 0, 0), thickness= -1) 
        
        c = (c + 1) & 7
        n = n + 1
        pd_list.append(d)
        if(n >= max_n_paralax):
            break
    img_rgb = cv2.cvtColor(img_dot, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(12, 8))
    plt.imshow(img_rgb)
    plt.xticks([])
    plt.yticks([])
    plt.show()

    if(flag_image_save):
        file_body = os.path.splitext(rds_file)[0]
        img_dot_file = file_body + "_dot.png"
        cv2.imwrite(img_dot_file, img_dot)
        
    for pd in pd_list:
        img_gray = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)
        # img_gamma = gamma_correction(img_gray, 0.8)
        img_b, img_g, img_r = cv2.split(img_src)
        img_b = image_affine(img_b   ,  pd/2, 0)
        img_g = image_affine(img_g   ,  pd/2, 0)
        img_r = image_affine(img_gray, -pd/2, 0)
        imgMerge_color = cv2.merge((img_b, img_g, img_r))
        imgMerge_rgb   = cv2.cvtColor(imgMerge_color, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12, 8))
        plt.imshow(imgMerge_rgb)
        plt.xticks([])
        plt.yticks([])
        plt.show()
    
        img_gray = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)
        img_b = image_affine(img_gray,  pd/2, 0)
        img_r = image_affine(img_gray, -pd/2, 0)
        height, width = img_gray.shape[:2]
        blank = np.zeros((height, width, 1), np.uint8)
        imgMerge_rb  = cv2.merge((img_b, blank, img_r))
        imgMerge_rgb = cv2.cvtColor(imgMerge_rb, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12, 8))
        plt.imshow(imgMerge_rgb)
        plt.xticks([])
        plt.yticks([])
        plt.show()
    
        if(flag_image_save):
            # img_right = image_affine(img_src,  pd/2, 0)
            # img_left  = image_affine(img_src, -pd/2, 0)
            # img_right_file = file_body + "_%03d_right.png" % (pd)
            # img_left_file  = file_body + "_%03d_left.png"  % (pd)
            # cv2.imwrite(img_right_file, img_right)
            # cv2.imwrite(img_left_file , img_left )
            img_color_file = file_body + "_%03d_color.png" % (pd)
            img_rb_file    = file_body + "_%03d_rb.png"    % (pd)
            cv2.imwrite(img_color_file, imgMerge_color)
            cv2.imwrite(img_rb_file   , imgMerge_rb)
    
cv2.destroyAllWindows()

